In [1]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

import os
import torch
import random
import json

from torch import nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from datetime import datetime
from collections import defaultdict, Counter
from tqdm import tqdm 

tqdm.pandas()

In [2]:
skills = pd.read_csv("../Data/skills_one-hot.csv").set_index("candidate_id")
skills.head()

,skill_1,skill_2,skill_3,skill_5,skill_6,skill_7,skill_8,skill_9,skill_12,skill_13,...,skill_3926,skill_3927,skill_3928,skill_3929,skill_3930,skill_3931,skill_3932,skill_3933,skill_3934,skill_3935
candidate_id,,,,,,,,,,,,,,,,,,,,,
84267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84349,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84381,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84432,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
skills = dict(zip(skills.index, skills.values))

In [4]:
certs = pd.read_csv("../Data/candidate_certificates_one-hot.csv").set_index("candidate_id")
certs.head()

,1,10,11,12,13,14,15,16,17,18,...,W4,W5,W7,W9,WB,WC,WD,WE,WF,ZW
candidate_id,,,,,,,,,,,,,,,,,,,,,
84603,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85035,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85214,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
certs = dict(zip(certs.index, certs.values))

In [6]:
w2v = json.load(open("../Data/embeddings.json"))
# Convert to ints
w2v = {int(k):{int(k2):v2 for k2, v2 in v.items()} for k, v in w2v.items()}

In [7]:
df_pred = pd.read_csv("../Data/df_pred.csv").drop("Unnamed: 0", axis=1)

In [8]:
df_pred.head()

,candidate_id,job_order,time_between,time_spent,isco_functie_niveau,education,function_id,isco_code4
0,84556,0,37,156,2.0,0.0,937.0,207.0
1,84556,1,23,116,1.0,0.0,811.0,347.0
2,84556,2,23,275,2.0,0.0,937.0,207.0
3,84556,3,1155,4,1.0,0.0,1521.0,343.0
4,84556,4,203,11,1.0,0.0,1521.0,343.0


In [9]:
career_paths = df_pred.groupby("candidate_id")

In [10]:
num_classes = len(df_pred["isco_code4"].unique())
num_features = len(career_paths.mean().columns)
num_classes, num_features

(354, 7)

In [11]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values[:,1:])

100%|██████████| 465468/465468 [00:45<00:00, 10280.12it/s]


In [12]:
# Drop careers that are longer than 100 jobs or only 1 job
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens <= 101) & (career_lens > 1)]

In [13]:
career_paths.head()

candidate_id
84556    [[0.0, 37.0, 156.0, 2.0, 0.0, 937.0, 207.0], [...
84612    [[0.0, 2537.0, 6.0, 1.0, 0.0, 1521.0, 343.0], ...
84731    [[0.0, 46.0, 23.0, 1.0, 0.0, 1521.0, 343.0], [...
85437    [[0.0, 747.0, 670.0, 1.0, 2.0, 1521.0, 343.0],...
85627    [[0.0, 140.0, 4627.0, 3.0, 0.0, 1922.0, 155.0]...
dtype: object

In [14]:
idxs = []
x = []
y = []

candidate_lens = defaultdict(int)

# max_skills = len([col for col in df_pred if "skill_" in col])

for idx, career in zip(career_paths.index, career_paths.values):
    label = career[-1, 6]
    
    if not np.isnan(label):
        candidate_lens[idx] = len(career) - 1
        
        idxs.append(idx)
        x.append(career[:-1].reshape(len(career) - 1, num_features))
        y.append(label)

idxs = np.array(idxs)
x = np.array(x)
y = np.array(y)

In [15]:
to_fill = np.zeros([len(x), len(max(x, key = lambda x: len(x))), num_features])

for i,j in enumerate(x):
    if len(j):
        to_fill[i][-len(j):] = j

In [16]:
max_len = len(max(x, key = lambda x: len(x)))

In [17]:
del df_pred
del x

In [18]:
device = ("cuda:0" if torch.cuda.is_available() else "cpu")
device

'cuda:0'

In [19]:
len(to_fill), len(y)

(265309, 265309)

In [20]:
# to_fill = to_fill[:75000]
# y = y[:75000]

In [20]:
# Train test split
split = 0.8
random.seed(42)

training = np.array(random.sample(range(len(to_fill)), int(split * len(to_fill))))
test = np.array(list(set(range(len(to_fill))) - set(training)))

train_indices, val_indices = idxs[training], idxs[test]
X_train, X_val = to_fill[training], to_fill[test]
y_train, y_val = y[training].astype(int), y[test].astype(int)

In [21]:
# Class weights
counts = np.bincount(y_train) + 1
labels_weights = 1. / counts
weights = labels_weights[y_train]
sampler = WeightedRandomSampler(weights, len(weights))

# Create dataloaders
train_data = TensorDataset(torch.Tensor(train_indices), 
                           torch.Tensor(X_train), 
                           torch.Tensor(y_train).type(torch.LongTensor))

trainloader = DataLoader(train_data, batch_size=512, sampler=sampler)

val_data = TensorDataset(torch.Tensor(val_indices),
                         torch.Tensor(X_val),
                         torch.Tensor(y_val).type(torch.LongTensor))

valloader = DataLoader(val_data, batch_size=512, shuffle=True)

In [22]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers, embedding_size, 
                 skills, certs, w2v, candidate_lengths, max_len):
        
        super(LSTM, self).__init__()
              
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size + 317 + 98 + 300
        self.hidden_size = hidden_size
        
        self.LSTMs = nn.ModuleList()
        
        for i in range(num_layers):
            input_size = self.input_size if i == 0 else hidden_size
            self.LSTMs.append(nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                                      num_layers=1, batch_first=True))
        
        self.fc = nn.Linear(hidden_size, num_classes)
        self.embedding = nn.Embedding(2, embedding_size)
        
        self.skill_keys = set(skills.keys())
        self.skills = np.vectorize(skills.get)
        
        self.certs_keys = set(certs.keys())
        self.certs = np.vectorize(certs.get)
        
        self.w2v_keys = set(w2v.keys())
        self.w2v = w2v
        
        self.candidate_lengths = candidate_lens
        self.max_len = max_len
        
        # self.conv = nn.Conv1d(613, 3, 1)

    def forward(self, candidate, x):       
        outputs = []
        
        # Default width of a row (filled with 0s)
        width = torch.Tensor([0] * (317 + 98 + 300)).type(torch.LongTensor).to(device)
        
        extra_features = []
                
        # For each candidate in the current batch
        for c in candidate:
            career_duration = self.candidate_lengths[c.item()]            
            
            # Look up skills            
            if c.item() in self.skill_keys:
                skill_list = torch.Tensor(self.skills(c.item())).type(torch.LongTensor).to(device)
            else:
                skill_list = torch.Tensor([0] * 317).type(torch.LongTensor).to(device)

            # Look up certificates
            if c.item() in self.certs_keys:
                certs_list = torch.Tensor(self.certs(c.item())).type(torch.LongTensor).to(device)
            else:
                certs_list = torch.Tensor([0] * 98).type(torch.LongTensor).to(device)
                
            ez_w2v = True
            w2v_list = torch.Tensor([0] * 300).type(torch.LongTensor).to(device)
            
            # Look for cvs
            if c.item() in self.w2v_keys:
                cvs = self.w2v[c.item()]
                
                storage = []
                
                 # If a candidate only has one CV, proceed as normal
                if len(cvs.keys()) == 1:
                    w2v_list = torch.Tensor(cvs[0]).type(torch.LongTensor).to(device)
                else: # Otherwise, stack them accordingly
                    ez_w2v = False
                    ks = np.array(list(cvs.keys()))
                    
                    # Find how many time steps (rows) each CV lasted
                    durations = [ks[i+1] - ks[i]
                                 if i < (len(ks) - 1) 
                                 else career_duration - ks[i]
                                 for i in range(len(ks))]
                                        
                    embed_values = list(cvs.values())
                    
                    # When the CV got updated on the last timestep, aka our test value
                    # Remove it from the list of durations, as it should be ignored
                    if durations[-1] == 0: 
                        durations.pop()
                    if durations[-1] == -1: # Sometimes contains -1 --> last location > (career duration)?
                        durations.pop()
                        durations[-1] -= 1
                        # In case the last one should be ignored completely
                        if durations[-1] == 0:
                            durations.pop()
                                               
                    if durations:
                        for i, duration in enumerate(durations):
                            storage.append(torch.stack([torch.Tensor(embed_values[i])] * duration, dim=0))
                    else:
                        w2v_list = torch.Tensor(cvs[0]).type(torch.LongTensor).to(device)

                    # Combine stored tensors into a single tensor
                    cv_embeddings = torch.cat((storage)).type(torch.LongTensor).to(device)
            else:
                w2v_list = torch.Tensor([0] * 300).type(torch.LongTensor).to(device)

            # Only create zeros if needed (e.g. less than max_len career duration)
            if (self.max_len - career_duration) > 0:
                zeros = torch.stack([width] * (self.max_len - career_duration))
            else: # Reset zeros to prevent shape mismatch
                zeros = torch.Tensor([]).type(torch.LongTensor).to(device)
                
            # If our word2vec is only one row like skills and certs
            if ez_w2v:
                # Fill a max_len row tensor with the correct values
                combined = torch.cat([skill_list, certs_list, w2v_list], dim=-1)
                
                # Combine the features
                filled_in = torch.stack([combined] * (career_duration))
            else: # Otherwise, we first broadcast it, and then concatenate it afterwards
                combined = torch.cat([skill_list, certs_list], dim=-1)
                sk_ct = torch.stack([combined] * career_duration)                                             
                filled_in = torch.cat([sk_ct, cv_embeddings], axis=1)
                
            total = torch.cat([zeros, filled_in], dim=0)
                    
            # Store result
            extra_features.append(total)
                                
        # Convert list of tensors to actual tensor
        additional_features = torch.stack((extra_features)).type(torch.FloatTensor).to(device)
                
        # Add features
        x = torch.cat([x, additional_features], dim=2)
        
        # Forward pass
        for i in range(self.num_layers):
            output, (h_n, c_n) = self.LSTMs[i](x)
            outputs.append(output)
            x = output

        h_out = h_n.view(-1, self.hidden_size)
        out = self.fc(h_out)
                        
        return out

In [23]:
def train_loop(model, trainloader, valloader, optimizer, criterion, num_epochs):

    results = defaultdict(list)
    
    # Train the model
    for epoch in range(num_epochs):
        for i, (candidate, career, job) in enumerate(trainloader):
            
            candidate, career, job = candidate.to(device), career.to(device), job.to(device)
            optimizer.zero_grad()

            outputs = model(candidate, career)

            # obtain the loss function
            loss = criterion(outputs, job)
            loss = loss.mean()           
            loss.backward()
            optimizer.step()
            
            print("Epoch: %d, batch: %d/%d, loss: %1.5f" % (epoch + 1, i + 1, len(trainloader), loss.item()), end="\r")
        
        stats = test_loop(valloader, model, criterion)
        results["Epoch"].append(epoch + 1)
        results["Acc@1"].append(stats[0])
        results["Acc@5"].append(stats[1])
        results["Acc@10"].append(stats[2])
        results["Acc@20"].append(stats[3])
        results["test_loss"].append(stats[4])
                
    return results
        
def test_loop(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, acc1, acc5, acc10, acc20 = 0, 0, 0, 0, 0
    
    with torch.no_grad():
        for candidate, career, job in dataloader:
            candidate, career, job = candidate.to(device), career.to(device), job.to(device)
            pred = model(candidate, career)
            
            test_loss += criterion(pred, job).mean().item()
            acc1 += (pred.argmax(1) == job).type(torch.float).sum().item()
            
            sorted_preds = torch.argsort(pred, 1, descending=True)
            
            acc5 += np.array([job[i].item() in sorted_preds[:,:5][i] for i in range(len(job))]).sum()
            acc10 += np.array([job[i].item() in sorted_preds[:,:10][i] for i in range(len(job))]).sum()
            acc20 += np.array([job[i].item() in sorted_preds[:,:20][i] for i in range(len(job))]).sum()

            
    # print("\nValidation:", Counter(np.array(pred.argmax(1).cpu())))
    test_loss /= num_batches
    acc1 /= size
    acc5 /= size
    acc10 /= size
    acc20 /= size
    print(f"\nTest Error:")
    print(f"Acc@1: {(100*acc1):>0.2f}%, Acc@5: {100*acc5:>0.2f}%, " +\
          f"Acc@10: {100*acc10:>0.2f}%, Acc@20: {100*acc20:>0.2f}% Avg loss: {test_loss:>8f}\n")
    
    return acc1, acc5, acc10, acc20, test_loss

In [ ]:
num_epochs = 20
current = 0

criterion = torch.nn.CrossEntropyLoss()

full_results = []

try:            
    for learning_rate in [1e-1, 1e-2, 1e-3, 1e-4][2:]:
        for num_layers in [1, 5, 10]:
            for hidden_size in [500, 1000][1:]:

                lstm = LSTM(num_classes=num_classes,
                            input_size=num_features,
                            num_layers=num_layers,
                            hidden_size=hidden_size,
                            embedding_size=5,
                            skills=skills, 
                            certs=certs,
                            w2v=w2v,
                            candidate_lengths=candidate_lens,
                            max_len=max_len)

                lstm = lstm.to(device)

                optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

                print(f"Current iteration {current}/{4**3}\n- Learning rate: {learning_rate}\n- Model: \n\n", lstm, "\n")

                # Store results of current configuration
                outcome = train_loop(lstm, trainloader, valloader, optimizer, criterion, num_epochs)
                outcome["lr"] = [learning_rate] * num_epochs
                outcome["Number of layers"] = [num_layers] * num_epochs
                outcome["Nodes per layer"] = [hidden_size] * num_epochs

                full_results.append(outcome)

                current += 1
            break

        # We ignore LR for now
        break
except KeyboardInterrupt:
    pass

Current iteration 0/64
- Learning rate: 0.001
- Model: 

 LSTM(
  (LSTMs): ModuleList(
    (0): LSTM(722, 200, batch_first=True)
  )
  (fc): Linear(in_features=200, out_features=354, bias=True)
  (embedding): Embedding(2, 5)
) 

Epoch: 1, batch: 415/415, loss: 4.20853
Test Error:
Acc@1: 7.14%, Acc@5: 21.44%, Acc@10: 30.33%, Acc@20: 41.53% Avg loss: 4.821265

Epoch: 2, batch: 415/415, loss: 3.80269
Test Error:
Acc@1: 11.31%, Acc@5: 27.13%, Acc@10: 35.81%, Acc@20: 47.28% Avg loss: 4.564900

Epoch: 3, batch: 415/415, loss: 3.36696
Test Error:
Acc@1: 11.23%, Acc@5: 28.07%, Acc@10: 37.35%, Acc@20: 49.45% Avg loss: 4.468983

Epoch: 4, batch: 415/415, loss: 3.17754
Test Error:
Acc@1: 13.19%, Acc@5: 31.88%, Acc@10: 41.90%, Acc@20: 54.31% Avg loss: 4.318421

Epoch: 5, batch: 415/415, loss: 3.02934
Test Error:
Acc@1: 14.13%, Acc@5: 33.32%, Acc@10: 43.61%, Acc@20: 55.71% Avg loss: 4.267223

Epoch: 6, batch: 415/415, loss: 2.86102
Test Error:
Acc@1: 12.81%, Acc@5: 32.27%, Acc@10: 43.04%, Acc@20: 5

In [ ]:
merge_results = defaultdict(list)

for res in full_results:
    for k, v in res.items():
        merge_results[k].extend(v)
        
total = pd.DataFrame(merge_results).set_index(["lr", "Number of layers", "Nodes per layer", "Epoch"])

In [ ]:
total

In [52]:
with torch.no_grad():
    for candidate, career, job in valloader:
        candidate, career, job = candidate.to(device), career.to(device), job.to(device)
        pred = lstm(candidate, career)
        
        print((pred.argmax(1) == job).type(torch.float).mean().item())        
        print(Counter(pred.argmax(1).tolist()))
        break

0.203125
Counter({331: 49, 343: 21, 348: 13, 176: 11, 163: 10, 184: 10, 350: 9, 300: 9, 335: 8, 341: 8, 322: 8, 342: 7, 351: 7, 179: 7, 196: 7, 155: 6, 333: 6, 231: 6, 193: 6, 347: 6, 173: 6, 321: 6, 344: 6, 213: 5, 117: 5, 214: 5, 177: 5, 327: 5, 273: 5, 326: 5, 264: 4, 203: 4, 2: 4, 238: 4, 221: 4, 98: 4, 325: 4, 79: 4, 338: 4, 206: 3, 289: 3, 200: 3, 71: 3, 271: 3, 187: 3, 83: 3, 143: 3, 156: 3, 232: 3, 137: 3, 305: 3, 207: 3, 175: 3, 192: 3, 129: 3, 24: 3, 190: 3, 168: 3, 307: 3, 153: 3, 61: 3, 46: 3, 261: 3, 144: 3, 194: 3, 182: 3, 208: 2, 114: 2, 107: 2, 4: 2, 6: 2, 224: 2, 239: 2, 254: 2, 150: 2, 145: 2, 290: 2, 62: 2, 164: 2, 186: 2, 336: 2, 95: 2, 77: 2, 345: 2, 92: 2, 103: 2, 269: 2, 222: 2, 274: 2, 188: 2, 252: 2, 174: 2, 139: 2, 178: 2, 74: 2, 233: 2, 189: 2, 240: 2, 235: 2, 265: 2, 70: 2, 91: 2, 282: 2, 97: 2, 73: 1, 283: 1, 89: 1, 136: 1, 162: 1, 260: 1, 110: 1, 205: 1, 140: 1, 181: 1, 118: 1, 5: 1, 285: 1, 180: 1, 72: 1, 201: 1, 12: 1, 141: 1, 352: 1, 195: 1, 14: 1, 9: 1

In [ ]:
emb = nn.Embedding(2, 4)
inpt = torch.Tensor([[1, 1, 1],
                     [0, 1, 0]]).type(torch.LongTensor)

emb(inpt).mean(axis=1)